In [1]:
!pip install vllm==0.6.0
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 MB 5.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.8 MB/s eta 0:00:0000:0100:11m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 4.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 8.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.

In [2]:
%cd /workspace/LLaMA-Factory/

/workspace/LLaMA-Factory


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from datasets import load_dataset
dataset = load_dataset("irene93/deidentification-chat-ko")

Dataset({
    features: ['instruction', 'input:', 'output'],
    num_rows: 940
})

In [5]:
import string
from vllm import LLM, SamplingParams

In [6]:
llm = LLM('./de-identification/model')

INFO 03-20 01:40:13 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='./de-identification/model', speculative_config=None, tokenizer='./de-identification/model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=./de-identification/model, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_output

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 03-20 01:40:53 model_runner.py:926] Loading model weights took 14.9595 GB
INFO 03-20 01:40:54 gpu_executor.py:122] # GPU blocks: 27978, # CPU blocks: 2048
INFO 03-20 01:40:58 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-20 01:40:58 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-20 01:41:18 model_runner.py:1335] Graph capturing finished in 20 secs.


In [7]:
from tqdm import tqdm

In [10]:
test_data = dataset['test']

In [11]:
test_data[0]

{'instruction': '고객: 국가유공자 우대 금리 정기예금에 가입하고 싶어요. 어떻게 하면 될까요?\n\n상담사: 어서오세요. 신한은행입니다. 어떤 방법으로 신청할 계획이신가요?\n\n고객: 인터넷 뱅킹을 활용하여 신청할 생각이에요.\n\n상담사: 인터넷 뱅킹을 사용해보신 적 있으신가요?\n\n고객: 네, 사용해본 경험이 있어요.\n\n상담사: 그렇다면, 신한은행 홈페이지에 접속하셔서 적금 가입 메뉴에서 선택하실 수 있습니다. 홈페이지의 안내를 따라 진행해보세요.\n\n고객: 네, 시도해보겠습니다.\n\n상담사: 감사합니다. 이상 신한은행 상담원 김민하 였습니다. 추가 문의 사항 있으시면 언제든지 연락 주세요.\n\n고객: 건물 보험을 해지하고 싶습니다.\n\n상담사: 네, 건물 보험 해지를 원하시는군요. 해지 절차를 도와드리겠습니다. 통화는 녹음되고 있음을 알려드립니다.\n\n고객: 네, 그렇게 하세요.\n\n상담사: 가입하신 보험의 식별 번호를 말씀해 주시면 감사하겠습니다.\n\n고객: 보험 번호 말씀하시는 건가요?\n\n상담사: 네, 보험을 구별하는 9자리의 고유번호입니다. 보험 증서 상단에서 확인할 수 있습니다.\n\n고객: 아, 여기 있네요. 123456789입니다.\n\n상담사: 확인 감사합니다. 지금 해지 절차를 진행하도록 하겠습니다. 그리고 고객님의 계좌번호를 확인해 주시면 환불 진행하겠습니다.\n\n고객: 제 국민은행 계좌번호는 110-1234-5678입니다.\n\n상담사: 감사합니다. 환불은 3일 이내로 처리될 예정입니다. 추가적으로 필요하신 사항이 있으시면 언제든지 문의 주시기 바랍니다.\n\n고객: 네, 잘 알았습니다.\n\n상담사: 행복한 하루 되세요. 이상 신한은행 김민하 였습니다. 고맙습니다.',
 'input:': '',
 'output': '고객: 국가유공자 우대 금리 정기예금에 가입하고 싶어요. 어떻게 하면 될까요?\n\n상담사: 어서오세요. 신한은행입니다. 어떤 방법으로 신청할 계획이신가요?\n\n고객: 인터넷 뱅킹을

In [84]:
compare_data =[]

940

In [85]:
# 프롬프트 템플릿 준비
template = string.Template("""<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

\n 뉴스데이터 :${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|> 
""")

system_prompt = '''당신은 개인정보를 감춰주는 로봇입니다. 
## 지시 사항 ##\n
1.주어진 대화에서 사람이름을 [PERSON1], [PERSON2] 등으로 등장 순서에 따라 대체하고, 동일한 이름이 반복될 경우 같은 대치어를 사용합니다.\n
2.연락처, 이메일, 주소 , 계좌번호도 각각 [CONTACT1], [CONTACT2] 등, [EMAIL1],[EMAIL2] 등, [ADDRESS1],[ADDRESS2]등 , [ACCOUNT1], [ACCOUNT2] 등 으로 대치하고 동일한 정보가 반복되는 경우에는 같은 대치어를 사용합니다.\n
3.대치어를 작성할때 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 \n
4.위 규칙은 대화 전체에 걸쳐 일관되게 적용합니다. \n
당신이 개인정보를 감출 대화내역입니다.
'''

for i in tqdm(range(440)):
    j = i + 500
    conv = test_data[j]
    # 메시지 준비
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": conv['instruction']}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })
        # 샘플링 파라미터 설정
    sampling_params = SamplingParams(
        temperature=0,
        max_tokens=1024,
        stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
        repetition_penalty=1.0
    )
    
    # 생성
    outputs = llm.generate([prompt], sampling_params)
    
    # 결과 출력
    result = ''.join([output.outputs[0].text for output in outputs])
    
    conv['sLLM_output'] = result
    compare_data.append(conv)

 25%|██▍       | 108/440 [09:05<27:56,  5.05s/it] ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


KeyboardInterrupt: 

In [86]:
len(compare_data)

608

In [87]:
import re
def extract_placeholder_mapping(original_text, transformed_text, allowed_types=("PERSON", "CONTACT", "ADDRESS", "EMAIL", "ACCOUNT")):
    """
    original_text: 변환 전 실제 텍스트 (여러 줄)
    transformed_text: 변환 후 placeholder가 포함된 텍스트 (여러 줄)
    allowed_types: 허용하는 placeholder 유형 (예, EMAIL은 제외됨)
    줄 단위로 처리하며, transformed 텍스트에서 모든 대괄호 그룹을 잡은 후,
    허용하는 placeholder에 해당하는 경우에만 해당 전후 literal을 이용해 원본의 치환된 실제 값을 추출합니다.
    """
    # 허용하는 placeholder만 매칭하는 정규식: 예: [PERSON1], [CONTACT2] 등
    allowed_pattern = re.compile(r'^\[(' + '|'.join(allowed_types) + r')(\d+)\]$')
    # 모든 대괄호 그룹을 잡는 패턴
    generic_pattern = re.compile(r'(\[[^]]+\])')
    mapping = {}
    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))
    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]
        # 모든 대괄호 그룹으로 분리합니다.
        parts = re.split(generic_pattern, trans_line)
        pos = 0  # 원본 라인에서 literal 검색 시작 인덱스
        # parts는 짝수 인덱스: literal, 홀수 인덱스: 대괄호 그룹(placeholder일 수도, 아닐 수도 있음)
        for i, part in enumerate(parts):
            if i % 2 == 0:
                # literal 부분: 원본에서 이 부분을 찾습니다.
                if part:
                    found_index = orig_line.find(part, pos)
                    if found_index == -1:
                        continue
                    # 직전에 나온 대괄호 그룹이 허용하는 placeholder라면,
                    # 원본에서 pos부터 found_index까지의 텍스트가 치환된 실제 값입니다.
                    if i > 0:
                        token = parts[i-1]
                        if allowed_pattern.match(token):
                            replaced_text = orig_line[pos:found_index].strip()
                            if replaced_text:
                                mapping[replaced_text] = token
                    pos = found_index + len(part)
            else:
                # 홀수 인덱스: 대괄호 그룹(placeholder 또는 그저 literal처럼 남은 부분)
                # 허용하지 않는 토큰은 그대로 literal에 포함되도록 처리합니다.
                pass
        # 마지막 literal 뒤에 남은 부분 처리 (대괄호 그룹이 마지막에 있었다면 이미 처리됨)
        if len(parts) % 2 == 1:
            replaced_text = orig_line[pos:].strip()
            # 마지막 토큰이 placeholder라면 처리하지만 보통 마지막 부분은 literal입니다.
            if replaced_text:
                # 만약 마지막 부분 앞 토큰이 허용하는 placeholder라면 매핑
                if len(parts) > 1:
                    token = parts[-2]
                    if allowed_pattern.match(token):
                        mapping[replaced_text] = token
    return mapping

In [58]:
org_conv = compare_data[0]['instruction']
openai_deid =  compare_data[0]['output']
sllm_deid = compare_data[0]['sLLM_output']

In [72]:
org_conv1 = compare_data[1]['instruction']
openai_deid1 =  compare_data[1]['output']
sllm_deid1 = compare_data[1]['sLLM_output']

In [73]:
extract_placeholder_mapping(org_conv, openai_deid)

{'김민하': '[PERSON1]', '123456789': '[CONTACT1]', '110-1234-5678': '[ACCOUNT1]'}

In [74]:
extract_placeholder_mapping(org_conv1, openai_deid1)

{'김현지': '[PERSON1]',
 '이주연': '[PERSON2]',
 '010-9876-5432': '[CONTACT1]',
 '서울특별시 강남구 역삼동 123-45': '[ADDRESS1]',
 'juyi@naver.com': '[EMAIL1]'}

In [88]:
sllm =[]
openai =[]
for d in  compare_data:
    org_conv = d['instruction']
    openai_deid =  d['output']
    sllm_deid = d['sLLM_output']

    openai_checked = extract_placeholder_mapping(org_conv, openai_deid)
    sllm_checked = extract_placeholder_mapping(org_conv, sllm_deid)
    sllm.append(sllm_checked)
    openai.append(openai_checked)

In [89]:
from collections import Counter

def compare_json_lists(json_list1, json_list2):
    """
    Compares two lists of JSON objects and calculates precision, recall, and F1-score
    based on key-value matches.
    """
    json1_items = set()
    json2_items = set()
    
    for json1 in json_list1:
        json1_items.update(json1.items())
    
    for json2 in json_list2:
        json2_items.update(json2.items())
    
    true_positives = len(json1_items & json2_items)
    false_positives = len(json2_items - json1_items)
    false_negatives = len(json1_items - json2_items)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "True Positives": true_positives,
        "False Positives": false_positives,
        "False Negatives": false_negatives,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score
    }


In [78]:
sllm[1]

{'상담사: 안녕하세요. 저는 삼성몰의 김현지': '[PERSON1]',
 '이주연': '[PERSON2]',
 '010-9876-5432': '[CONTACT1]',
 '서울특별시 강남구 역삼동 123-45': '[ADDRESS1]',
 'juyi@naver.com': '[EMAIL1]',
 '김현지': '[PERSON1]'}

In [90]:
from collections import Counter

def compare_json_lists_values(json_list1, json_list2):
    """
    Compares two lists of JSON objects and calculates precision, recall, and F1-score
    based on key-value matches.
    """
    json1_dict = {}
    json2_dict = {}
    
    for json1 in json_list1:
        json1_dict.update(json1)
    
    for json2 in json_list2:
        json2_dict.update(json2)
    
    true_positives = sum(1 for k in json1_dict if k in json2_dict and json1_dict[k] == json2_dict[k])
    false_positives = sum(1 for k in json2_dict if k not in json1_dict or json1_dict[k] != json2_dict[k])
    false_negatives = sum(1 for k in json1_dict if k not in json2_dict or json1_dict[k] != json2_dict[k])
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "True Positives": true_positives,
        "False Positives": false_positives,
        "False Negatives": false_negatives,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score
    }


In [93]:
compare_json_lists_values(sllm, openai)

{'True Positives': 774,
 'False Positives': 143,
 'False Negatives': 170,
 'Precision': 0.8440567066521265,
 'Recall': 0.8199152542372882,
 'F1 Score': 0.8318108543793659}

In [82]:
json1_dict = {}
json2_dict = {}

for json1 in sllm:
    json1_dict.update(json1)

for json2 in openai:
    json2_dict.update(json2)

In [83]:
json1_dict

{'김민하': '[PERSON1]',
 '123456789': '[CONTACT2]',
 '110-1234-5678': '[ACCOUNT1]',
 '상담사: 안녕하세요. 저는 삼성몰의 김현지': '[PERSON1]',
 '이주연': '[PERSON2]',
 '010-9876-5432': '[CONTACT1]',
 '서울특별시 강남구 역삼동 123-45': '[ADDRESS1]',
 'juyi@naver.com': '[EMAIL1]',
 '김현지': '[PERSON1]',
 '김미영': '[PERSON1]',
 '서울특별시 강남구 테헤란로 123번지': '[ADDRESS1]',
 '박지혜': '[PERSON1]',
 '010-1234-5678': '[CONTACT1]',
 '서울특별시 강동구 천호동 123-45번지': '[ADDRESS1]',
 '국민은행 987654-12-345678': '[ACCOUNT1]',
 '이수민': '[PERSON1]',
 '서울특별시 강남구 테헤란로 123': '[ADDRESS1]',
 '1234-5678-9876-5432': '[ACCOUNT1]',
 '이수진': '[PERSON2]',
 '010-3456-7890': '[CONTACT1]',
 '서울특별시 강남구 삼성동 123-45번지, 201호': '[ADDRESS1]',
 '110-4567-8910': '[ACCOUNT1]',
 '김민수': '[PERSON1]',
 '서울시 강남구 청담동 101-202': '[ADDRESS1]',
 '123-45-678901': '[ACCOUNT1]',
 '123-456-789012': '[ACCOUNT1]',
 '서울시 강남구 테헤란로123': '[ADDRESS1]',
 '6789-1234-5678-0000': '[ACCOUNT1]',
 '이지수': '[PERSON2]',
 '01012345678': '[CONTACT1]',
 '서울특별시 은평구 녹번동 201-34번지': '[ADDRESS1]',
 '890101-1234567': '[CON